In [1]:
# pip install python-dotenv
# pip install spotipy --upgrade
# pip install python-vlc
import os
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import spotipy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import vlc
import json

load_dotenv()

client_credentials_manager = SpotifyClientCredentials(client_id=os.environ.get("SPOTIPY_CLIENT_ID"),
                                                      client_secret=os.environ.get("SPOTIPY_CLIENT_SECRET"))

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# https://open.spotify.com/playlist/37i9dQZF1E38k2WiBlq4DF?si=96e4439556ff412c
# https://open.spotify.com/track/0rc3gOGiTLUz9rZ3ypsYW7?si=721667e208cb4c64
# results = sp.playlist("37i9dQZF1E38k2WiBlq4DF")
# results = sp.track("66lF1QWtLZXJofJ3x388RV")

In [2]:
def get_genres_by_track_id(track_id):

    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET))

    track_data = sp.track(track_id)

    artist_ids = []

    for artist in track_data["artists"]:
        artist_ids.append(artist["id"])

    artists_data = sp.artists(artist_ids)

    genres = []

    for artist in artists_data["artists"]:
        genres += artist["genres"]
        break

    return genres

def play_music(final_df):
    #ลองเล่นเพลงที่ได้จาก preview_url
    p.stop()
    song = str(final_df['preview_url'].sample().values[0])
    p = vlc.MediaPlayer(song)

    #ตั้งความดังเสียง
    p.audio_set_volume(50)

    p.play()

def get_genres():
    # Open genres file
    try:
        with open('genres.json', 'r') as infile:
            valid_genres = json.load(infile)
    except FileNotFoundError:
        print("Couldn't find genres file!")
        sys.exit(1)
    
    return valid_genres

In [102]:
genres_list = get_genres()
ids=[]
song_meta={'id':[],'preview_url':[],'genre':[]}
limit_per_genres = 2

In [67]:
len(genres_list)

126

In [103]:
for genre in genres_list:
    index = len(ids)
    while list(song_meta.items())[2][1].count(genre) < limit_per_genres:
        
        results = sp.search(q="genre:"+genre)

        # create a list of song ids
        for item in results['tracks']['items']:
            print(genre)
            if item['id'] in ids:
                continue
            track = item['id']
            ids.append(track)

        for song_id in ids[index:]:

            if list(song_meta.items())[2][1].count(genre) == limit_per_genres:
                break
                
            # get song's meta data
            meta = sp.track(song_id)

            # song id
            song_meta['id'].append(song_id)

            # song preview
            preview_url = meta['preview_url']
            song_meta['preview_url'] += [preview_url]

            # song genres
            song_genre = genre
            song_meta["genre"] += [song_genre]

In [16]:
len(ids)

152030660

In [ ]:
song_meta_df = pd.DataFrame.from_dict(song_meta)
 
# check the song feature
features = sp.audio_features(song_meta['id'])

# change dictionary to dataframe
features_df = pd.DataFrame.from_dict(features)

# combine two dataframe
final_df = song_meta_df.merge(features_df)

In [ ]:
final_df.head()

In [ ]:
play_music(final_df)

In [ ]:
final_df.columns

In [ ]:
final_df = final_df.drop(columns=['type','uri', 'track_href', 'analysis_url','duration_ms', 'time_signature'])

In [ ]:
final_df.to_csv()